In [22]:
!pip install scapy

DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from scapy.all import * # Packet manipulation
import pandas as pd # Pandas - Create and Manipulate DataFrames
import numpy as np # Math Stuff (don't worry only used for one line :] )
import binascii # Binary to Ascii
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline

Preámbulo

In [24]:
# Capturando los primeros 25 paquetes de la red doméstica
captured_packets = sniff(count=25)

# Imprimiendo tipo de variable, longitud y contenido
print("Tipo de variable:", type(captured_packets))
print("Longitud de la variable:", len(captured_packets))
print("Contenido de la variable:", captured_packets)

# Imprimiendo el tipo de dato del primer paquete capturado
first_packet = captured_packets[0]
print("Tipo de dato del primer paquete capturado:", type(first_packet))

# Imprimiendo contenido de 5 paquetes
print("\nContenido de los primeros 5 paquetes capturados:")
for i in range(5):
    print("Paquete", i+1, ":", captured_packets[i].summary())

Tipo de variable: <class 'scapy.plist.PacketList'>
Longitud de la variable: 25
Contenido de la variable: <Sniffed: TCP:16 UDP:9 ICMP:0 Other:0>
Tipo de dato del primer paquete capturado: <class 'scapy.layers.l2.Ether'>

Contenido de los primeros 5 paquetes capturados:
Paquete 1 : Ether / IP / TCP 192.168.5.206:52978 > 52.226.139.180:https A / Raw
Paquete 2 : Ether / IP / TCP 52.226.139.180:https > 192.168.5.206:52978 A
Paquete 3 : Ether / IP / UDP 0.0.0.0:11113 > 255.255.255.255:11111 / Raw
Paquete 4 : Ether / IP / TCP 162.159.130.234:https > 192.168.5.206:43580 PA / Raw
Paquete 5 : Ether / IP / TCP 192.168.5.206:43580 > 162.159.130.234:https A


Análisis estadístico

In [25]:
paquete_pcap = rdpcap("analisis_paquetes.pcap")

df = pd.DataFrame()

# Iterando sobre los paquetes en paquete_pcap
for i, packet in enumerate(paquete_pcap):
    # Verificar si el paquete tiene la capa IP
    if IP in packet:
        # Si la capa IP existe, se accede a los campos 'src' y 'dst'
        src_address = packet[IP].src
        dst_address = packet[IP].dst
    else:
        src_address = None
        dst_address = None

    # Verificando si el paquete tiene la capa TCP o UDP
    if TCP in packet:
        # Si la capa TCP existe, se accede a los campos 'sport' y 'dport'
        src_port = packet[TCP].sport
        dst_port = packet[TCP].dport
    elif UDP in packet:
        # Si la capa UDP existe, se accede a los campos 'sport' y 'dport'
        src_port = packet[UDP].sport
        dst_port = packet[UDP].dport
    else:
        src_port = None
        dst_port = None

    # Añadiendo los campos al DataFrame
    df_temp = pd.DataFrame({'Src Address': [src_address], 'Dst Address': [dst_address], 'Src Port': [src_port], 'Dst Port': [dst_port]})
    df = pd.concat([df, df_temp], ignore_index=True)

print(df.head())

   Src Address  Dst Address  Src Port  Dst Port
0   10.1.10.53  84.54.22.33        53        53
1  84.54.22.33   10.1.10.53        53        53
2   10.1.10.53  84.54.22.33        53        53
3  84.54.22.33   10.1.10.53        53        53
4   10.1.10.53  84.54.22.33        53        53


Estadísticas

Inciso a. Muestre todas las IP origen

In [26]:
print("Todas las IP origen:")
print(df['Src Address'].unique())

Todas las IP origen:
['10.1.10.53' '84.54.22.33' '75.75.75.75']


Inciso b. Muestre todas las IP destino

In [27]:
print("\nTodas las IP destino:")
print(df['Dst Address'].unique())


Todas las IP destino:
['84.54.22.33' '10.1.10.53' '75.75.75.75']


Inciso c.

In [28]:
# c. ¿Cuál es la IP origen más frecuente?
src_ip_mas_frecuente = df['Src Address'].value_counts().idxmax()
print("\nLa IP origen más frecuente es:", src_ip_mas_frecuente)

# i. ¿A qué IP destino se comunica con más frecuencia?
dst_ip_mas_frecuente = df[df['Src Address'] == src_ip_mas_frecuente]['Dst Address'].value_counts().idxmax()
print("La IP destino con la que se comunica más frecuentemente es:", dst_ip_mas_frecuente)

# ii. ¿A que puerto destino se comunica? ¿Cuál es el propósito de este puerto?
dst_port_mas_frecuente = df[(df['Src Address'] == src_ip_mas_frecuente) & (df['Dst Address'] == dst_ip_mas_frecuente)]['Dst Port'].value_counts().idxmax()
print("El puerto destino con el que se comunica más frecuentemente es:", dst_port_mas_frecuente)
print("El propósito de este puerto puede variar dependiendo del protocolo y la aplicación que lo esté utilizando.")

# iii. ¿Desde que puertos origen se comunica?
src_ports = df[df['Src Address'] == src_ip_mas_frecuente]['Src Port'].unique()
print("\nLos puertos origen desde los que se comunica son:", src_ports)


La IP origen más frecuente es: 10.1.10.53
La IP destino con la que se comunica más frecuentemente es: 84.54.22.33
El puerto destino con el que se comunica más frecuentemente es: 53
El propósito de este puerto puede variar dependiendo del protocolo y la aplicación que lo esté utilizando.

Los puertos origen desde los que se comunica son: [   53 15812 23903]
